In [ ]:
import pandas as pd
import os
import zipfile
import time
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
new_folder_path = '/content/dataset'
os.makedirs(new_folder_path, exist_ok=True)

zip_file_path = '/content/drive/MyDrive/MLMA Project/GROUPBY SPLIT/Use This Final Final (22 04 25)/augumented_final.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(new_folder_path)

print(f"File '{zip_file_path}' unzipped to '{new_folder_path}'")

File '/content/drive/MyDrive/MLMA Project/GROUPBY SPLIT/Use This Final Final (22 04 25)/augumented_final.zip' unzipped to '/content/dataset'


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

train_dir = "/content/dataset/augumented_final/train"
valid_dir = "/content/dataset/augumented_final/valid"
test_dir  = "/content/dataset/augumented_final/test"

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


# Load train, validation and test sets
batch_size = 32
num_workers = 4

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=val_test_transform)
test_dataset  = datasets.ImageFolder(root=test_dir,  transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=num_workers)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=num_workers)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)

# Compute class weights
train_labels = [label for _, label in train_dataset.samples]
class_counts = np.bincount(train_labels, minlength=num_classes)
class_weights = 1.0 / class_counts
class_weights = class_weights * (num_classes / class_weights.sum())
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
print("Computed class weights:", class_weights)

# Function to train the ResNet50 model
def Train_model(num_epochs, model, train_loader, valid_loader, criterion,
                optimizer, patience=3, ckpt_dir="checkpoints"):
    """
    Trains the model with early stopping and saves the best checkpoint.
    Returns the best model (loaded with best weights) and its validation accuracy.
    """
    os.makedirs(ckpt_dir, exist_ok=True)
    model.to(device)

    best_val_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    trigger_times = 0

    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Train)"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        train_loss = running_loss / total_train
        train_acc  = 100 * correct_train / total_train
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

        # Validation loop
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for images, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Valid)"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                running_val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)

        val_loss = running_val_loss / total_val
        val_acc  = 100 * correct_val / total_val
        print(f"Epoch {epoch+1}, Valid Loss: {val_loss:.4f}, Valid Acc: {val_acc:.2f}%")

        # Early stopping & checkpoint
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            trigger_times = 0

            ckpt_path = os.path.join(
                ckpt_dir,
                f"best_model_epoch{epoch+1:02d}_acc{val_acc:.2f}.pth"
            )
            torch.save(model.state_dict(), ckpt_path)
            print(f"  → Saved new best model to {ckpt_path}")
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Load best weights
    model.load_state_dict(best_model_wts)
    return model, best_val_acc


# Hyperparameter grid search
learning_rates = [1e-3, 5e-4]
weight_decays  = [0.0,   1e-4]
num_epochs     = 10

best_val_acc = 0.0
best_config  = None
best_model   = None

print("\nStarting ResNet50 hyperparameter grid search...\n")
for lr in learning_rates:
    for wd in weight_decays:
        print(f"→ lr={lr}, weight_decay={wd}")

        # Initialize pretrained ResNet50 model
        model = models.resnet50(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

        # Loss & optimizer
        criterion = nn.CrossEntropyLoss(weight=class_weights)
        optimizer = optim.Adam(
            model.parameters(),
            lr=lr, weight_decay=wd
        )

        # Train with early stopping
        trained_model, val_acc = Train_model(
            num_epochs,
            model,
            train_loader,
            valid_loader,
            criterion,
            optimizer,
            patience=3,
            ckpt_dir="checkpoints"
        )

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_config  = (lr, wd)
            best_model   = copy.deepcopy(trained_model)

print("\n==> Best Configuration:")
print(f"   Learning Rate: {best_config[0]}")
print(f"   Weight Decay:   {best_config[1]}")
print(f"   Val Accuracy:   {best_val_acc:.2f}%")

# Save the best model
final_path = os.path.join(
    "checkpoints",
    f"final_best_lr{best_config[0]}_wd{best_config[1]}_acc{best_val_acc:.2f}.pth"
)
torch.save(best_model.state_dict(), final_path)
print(f"Final best model saved to {final_path}")

Using device: cuda
Classes: ['A', 'C', 'D', 'G', 'H', 'M', 'N', 'O']
Computed class weights: tensor([1.2101, 1.0815, 0.1979, 1.1540, 2.4601, 1.3247, 0.1110, 0.4606],
       device='cuda:0')

Starting ResNet50 hyperparameter grid search...

→ lr=0.001, weight_decay=0.0


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 207MB/s]
Epoch 1/10 (Train): 100%|██████████| 280/280 [00:15<00:00, 17.64it/s]


Epoch 1, Train Loss: 1.7992, Train Acc: 19.70%


Epoch 1/10 (Valid): 100%|██████████| 41/41 [00:01<00:00, 36.17it/s]


Epoch 1, Valid Loss: 2.2696, Valid Acc: 9.52%
  → Saved new best model to checkpoints/best_model_epoch01_acc9.52.pth


...

Epoch 6, Valid Loss: 1.8192, Valid Acc: 27.46%


Epoch 7/10 (Train): 100%|██████████| 280/280 [00:14<00:00, 19.24it/s]


Epoch 7, Train Loss: 0.6737, Train Acc: 52.23%


Epoch 7/10 (Valid): 100%|██████████| 41/41 [00:01<00:00, 37.08it/s]


Epoch 7, Valid Loss: 2.0536, Valid Acc: 33.54%
Early stopping at epoch 7

==> Best Configuration:
   Learning Rate: 0.0005
   Weight Decay:   0.0
   Val Accuracy:   44.70%
Final best model saved to checkpoints/final_best_lr0.0005_wd0.0_acc44.70.pth


In [ ]:
# Download checkpoints
!zip -r /content/checkpoints.zip /content/checkpoints
files.download("/content/checkpoints.zip")

  adding: content/checkpoints/ (stored 0%)
  adding: content/checkpoints/best_model_epoch03_acc33.62.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch02_acc38.46.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch07_acc44.70.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch01_acc15.37.pth (deflated 7%)
  adding: content/checkpoints/final_best_lr0.0005_wd0.0_acc44.70.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch02_acc30.66.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch04_acc37.36.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch05_acc33.00.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch01_acc9.52.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch06_acc33.46.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch03_acc32.22.pth (deflated 7%)
  adding: content/checkpoints/best_model_epoch02_acc27.61.pth (deflated 7%)
  adding: content/checkpoints/best_mode

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
save_folder = "saved_models"
os.makedirs(save_folder, exist_ok=True)
save_filename = "best_model_weights.pth"
save_path = os.path.join(save_folder, save_filename)
torch.save(best_model.state_dict(), save_path)  # Save the best model
print(f"Model weights saved to {save_path}")

Model weights saved to saved_models/best_model_weights.pth


In [ ]:
# Evaluate the best model on the test set

best_model.eval()

correct    = 0
total      = 0
all_preds  = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs  = inputs.to(device)
        labels  = labels.to(device)

        outputs  = best_model(inputs)
        _, preds = torch.max(outputs, 1)

        correct  += torch.sum(preds == labels).item()
        total    += labels.size(0)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = correct / total
print(f"\nTest Accuracy: {test_accuracy:.4f}")

# Compute metrics by each class
precision_pc, recall_pc, f1_pc, support_pc = precision_recall_fscore_support(
    all_labels,
    all_preds,
    average=None
)

print("\nPer-class Recall:")
for cls_name, rec, sup in zip(train_dataset.classes, recall_pc, support_pc):
    print(f"  {cls_name}: {rec:.4f}  (support={sup})")

weights = support_pc / support_pc.sum()
print("\nWeighted Recall Contributions:")
for cls_name, rec, w in zip(train_dataset.classes, recall_pc, weights):
    print(f"  {cls_name}: recall={rec:.4f}, weight={w:.4f}, contribution={rec*w:.4f}")

total_weighted_recall = np.dot(recall_pc, weights)

# weighted metrics
precision_w, recall_w, f1_w, _ = precision_recall_fscore_support(
    all_labels,
    all_preds,
    average='weighted'
)

# Print results
print("\nWeighted Precision, Recall, F1:")
print(f"  Weighted Precision: {precision_w:.4f}")
print(f"  Weighted Recall:    {recall_w:.4f}")
print(f"  Weighted F1-score:  {f1_w:.4f}")

# classification report
print("\nClassification Report:")
print(classification_report(
    all_labels,
    all_preds,
    target_names=train_dataset.classes,
    digits=4
))


Test Accuracy: 0.4485

Per-class Recall:
  A: 0.5862  (support=29)
  C: 1.0000  (support=23)
  D: 0.0741  (support=162)
  G: 0.2414  (support=29)
  H: 0.0000  (support=9)
  M: 0.7500  (support=16)
  N: 0.6976  (support=291)
  O: 0.1250  (support=72)

Weighted Recall Contributions:
  A: recall=0.5862, weight=0.0460, contribution=0.0269
  C: recall=1.0000, weight=0.0365, contribution=0.0365
  D: recall=0.0741, weight=0.2567, contribution=0.0190
  G: recall=0.2414, weight=0.0460, contribution=0.0111
  H: recall=0.0000, weight=0.0143, contribution=0.0000
  M: recall=0.7500, weight=0.0254, contribution=0.0190
  N: recall=0.6976, weight=0.4612, contribution=0.3217
  O: recall=0.1250, weight=0.1141, contribution=0.0143

Weighted Precision, Recall, F1:
  Weighted Precision: 0.4710
  Weighted Recall:    0.4485
  Weighted F1-score:  0.3995

Classification Report:
              precision    recall  f1-score   support

           A     0.2073    0.5862    0.3063        29
           C     0.3651 